In [40]:
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
from perplexity import Perplexity



In [41]:
pip show perplexityai

Name: perplexityai
Version: 0.17.0
Summary: The official Python library for the perplexity API
Home-page: https://github.com/perplexityai/perplexity-py
Author: 
Author-email: Perplexity <api@perplexity.ai>
License: Apache-2.0
Location: c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages
Requires: anyio, distro, httpx, pydantic, sniffio, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [54]:

# ---------- FUNCTIONS ----------

def get_pdf_text(pdf_path):
    """Extract text from a single PDF file"""
    text = ""
    pdf_reader = PdfReader(pdf_path)
    for page in pdf_reader.pages:
        text += page.extract_text()
    print("\n🤔 pdf read success!!")

    return text


def get_text_chunks(text):
    """Split text into manageable chunks"""
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    print("\n🤔 text converted into chunks!!")

    return text_splitter.split_text(text)


def create_vector_store(text_chunks):
    """Store text chunks in FAISS (no embeddings API needed)"""
    # We'll just use simple FAISS indexing for similarity search
    # For better results, you could use sentence-transformers locally
    from sentence_transformers import SentenceTransformer
    from langchain.embeddings import HuggingFaceEmbeddings

    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
    print("\n🤔 vectorization of chunks successfully done!!")

    return FAISS.from_texts(text_chunks, embeddings)


def ask_perplexity(question, api_key):
    # """Ask Perplexity API a question"""
    # url = "https://api.perplexity.ai/answers"
    # headers = {
    #     "Authorization": f"Bearer {api_key}",
    #     "Content-Type": "application/json"
    # }
    # data = {
    #     "question": question,
    #     "num_results": 1  # get best answer
    # }
    # response = requests.post(url, json=data, headers=headers)
    # response.raise_for_status()
    # answer_data = response.json()
    # # Extract the answer text
    # return answer_data.get("answer") or answer_data.get("answers", [{}])[0].get("text", "No answer found")
    

# Initialize client with explicit API key
    # print(question)
    client = Perplexity(api_key=os.environ.get("PERPLEXITY_API_KEY"))

    completion = client.chat.completions.create(
        model="sonar-pro",
        messages=[
            {"role": "user", "content": question}
        ]
    )

    print(completion.choices[0].message.content)
    return completion.choices[0].message.content
    


def ask_question(vector_store, question, api_key):
    """Retrieve relevant chunks and ask Perplexity API"""
    docs = vector_store.similarity_search(question, k=3)
    context_text = "\n\n".join([doc.page_content for doc in docs])
    combined_question = f"Context:\n{context_text}\n\nQuestion: {question}"

    return ask_perplexity(combined_question, api_key)

In [ ]:

def main():
    load_dotenv()
    api_key = os.getenv("PERPLEXITY_API_KEY")  # Your Airtel free Pro API key

    if not api_key:
        print("❌ Missing PERPLEXITY_API_KEY in .env file.")
        return

    # Ask for the PDF file
    pdf_path = "./test.pdf"

    if not os.path.exists(pdf_path):
        print(f"❌ File '{pdf_path}' not found.")
        return

    print("\n📚 Reading and processing PDF...")
    raw_text = get_pdf_text(pdf_path)
    text_chunks = get_text_chunks(raw_text)
    vector_store = create_vector_store(text_chunks)

    # Ask the user for a question
    question = "General Rules and Regulations "

    print("\n🤔 Thinking...\n")

    try:

        answer = ask_question(vector_store, question, api_key)
        print("✨ Answer:\n")
        print(answer)
        print("✨ questions:\n",question)
    except Exception as e:
        print(f"❌ Error: {e}")



In [60]:
main()


📚 Reading and processing PDF...

🤔 pdf read success!!

🤔 text converted into chunks!!

🤔 vectorization of chunks successfully done!!

🤔 Thinking...

✨ questions:
 General Rules and Regulations 
General college rules and regulations establish clear expectations for conduct, procedures, and responsibilities for both students and staff. According to your provided context, key general rules include:

- **Authority and Inspection**: Security personnel or designated staff can inspect vehicle records and record details in a gate register.

- **Complaints and Grievances**: Student complaints must follow the proper channel and should use the institution's grievance redress mechanism.

- **Rule Changes**: The college reserves the right to amend or modify rules as needed to meet changing requirements. Rules are subject to revision, and updated information is officially communicated. Ignorance of these updates is not an excuse for non-compliance.

- **Notices and Announcements**: All students and

In [46]:
try:
    print(question)
    answer = ask_question(vector_store, question, api_key)
    print("✨ Answer:\n")
    print(answer)
except Exception as e:
    print(f"❌ Error: {e}")

What is the capital of Kerala?
❌ Error: 'list' object has no attribute 'similarity_search'


In [33]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("PERPLEXITY_API_KEY")

question = "What is the capital of Kerala?"

url = "https://api.perplexity.ai/answers"
headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
data = {"question": question, "num_results": 1}

response = requests.post(url, json=data, headers=headers)
answer_data = response.json()
print("Q:", question)
print("A:", answer_data.get("answer") or answer_data.get("answers", [{}])[0].get("text", "No answer found"))


JSONDecodeError: Expecting value: line 1 column 1 (char 0)